In [1]:
! pip install geopandas
! pip install pandas
! pip install pygeos
! pip install boto
! pip install s3fs
! pip install shapely
! pip install gcsfs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gzip
import time
from itertools import groupby
import geopandas as gpd

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_93706/1029240406.py:14: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).


In [ ]:
#UPDATE COLUMNS ON THE TABLES
scenarios = ['Aug2021','Aug2020','Baseline2019','Jan2022','May2020','May2022']
for scenario in scenarios:
    print('Read {}'.format(scenario))
    nuovo  = pd.read_csv('outputs/MergedPTs_NYC_{}.csv.gz'.format(scenario), index_col=0)
    nuovo2 =  pd.read_csv('outputs/MergedPtoPTs_NYC_{}.csv.gz'.format(scenario), index_col=0)
    nuovo = nuovo[[ 'vehicle', 'mode', 'startX', 'startY', 'endX', 'endY',
       'vehicleType', 'arrivalTime', 'departureTime', 'primaryFuel',
       'primaryFuelType', 'secondaryFuel', 'secondaryFuelType', 'capacity',
       'riders', 'duration', 'occupancy', 'vehicleMiles',
       'passengerMiles', 'totalEnergyInJoules', 'gallonsGasoline']]
    nuovo2 = nuovo2[['vehicleID', 'pathTraversalID', 'personID', 'planIndex', 'agency']]
    new_list_riders = []
    for list_riders in nuovo['riders']:
        new_list = ''
        for rider in list_riders.split(','):
            new_list+=rider+':'
        new_list_riders.append(new_list[1:-2])
    nuovo['riders'] = new_list_riders
    nuovo['riders'].replace('', None)
    nuovo.to_csv('outputs/MergedPTs_NYC_{}_2.csv.gz'.format(scenario))
    nuovo2.to_csv('outputs/MergedPtoPTs_NYC_{}_2.csv.gz'.format(scenario))

In [ ]:
#UPDATE COLUMNS ON THE TABLES 2
scenarios = ['Aug2021','Aug2020','Baseline2019','Jan2022','May2020','May2022']
for scenario in scenarios:
    print('Read {}'.format(scenario))
    nuovo  = pd.read_csv('outputs/MergedPTs_NYC_{}_2.csv.gz'.format(scenario), index_col=0)
    nuovo2 =  pd.read_csv('outputs/MergedPtoPTs_NYC_{}_2.csv.gz'.format(scenario), index_col=0)
    nuovo['driver'] = False
    nuovo['mode_extended'] = False

    nuovo.to_csv('outputs/MergedPTs_NYC_{}_3.csv.gz'.format(scenario))
    nuovo2.to_csv('outputs/MergedPtoPTs_NYC_{}_3.csv.gz'.format(scenario))

In [ ]:
#MERGE POPULATION AND HOUSEHOLDS

fp = "s3://beam-outputs/output/newyork/"
output_nm = 'Baseline'
fp_res = 'outputs/'

data = {
    'new-york-baseline-0-of-10__2023-01-03_19-59-12_nwn/ITERS/it.5/5.events.csv.gz':'baseline-0',
    'new-york-baseline-1-of-10__2023-01-03_19-59-13_jgp/ITERS/it.5/5.events.csv.gz':'baseline-1',
    'new-york-baseline-2-of-10__2023-01-03_19-59-06_brm/ITERS/it.5/5.events.csv.gz':'baseline-2',
    'new-york-baseline-3-of-10__2023-01-03_19-59-07_kqr/ITERS/it.5/5.events.csv.gz':'baseline-3',
    'new-york-baseline-4-of-10__2023-01-03_19-59-09_fbb/ITERS/it.5/5.events.csv.gz':'baseline-4',
    'new-york-baseline-5-of-10__2023-01-03_19-59-11_tjh/ITERS/it.5/5.events.csv.gz':'baseline-5',
    'new-york-baseline-6-of-10__2023-01-03_19-59-12_zwm/ITERS/it.5/5.events.csv.gz':'baseline-6',
    'new-york-baseline-7-of-10__2023-01-03_19-59-19_bcr/ITERS/it.5/5.events.csv.gz':'baseline-7',
    'new-york-baseline-8-of-10__2023-01-03_19-59-14_olx/ITERS/it.5/5.events.csv.gz':'baseline-8',
    'new-york-baseline-9-of-10__2023-01-03_19-59-07_xcc/ITERS/it.5/5.events.csv.gz':'baseline-9',
}
data_names = data.keys()
pop_names = []
hs_names = []
for data_name in data_names:
    pop_names.append(data_name[:-26]+'population.csv.gz')
    hs_names.append(data_name[:-26]+'households.csv.gz')
names = data.values()

Merged_Pop = []
Merged_Hs = []
for name, pop_name, hs_name in zip(names, pop_names, hs_names):
    Merged_Pop.append(pd.read_csv(fp+pop_name)) 
    Merged_Hs.append(pd.read_csv(fp+hs_name))
Merged_Pop = pd.concat(Merged_Pop)
Merged_Hs = pd.concat(Merged_Hs)
Merged_Pop.to_csv(fp_res+'Merged_Pop_{}.csv.gz'.format(output_nm))
Merged_Hs.to_csv(fp_res+'Merged_Hs_{}.csv.gz'.format(output_nm))
print(len(Merged_Pop))
print(len(Merged_Hs))

In [ ]:
fp = "s3://beam-outputs/output/newyork/"
output_nm = 'Jan2022'
fp_res = 'outputs/'

data = {
    'new-york-jan2022-0-of-10__2023-01-04_00-19-31_leg/ITERS/it.5/5.events.csv.gz':'jan2022-0',
    'new-york-jan2022-1-of-10__2023-01-14_04-04-57_ztd/ITERS/it.5/5.events.csv.gz':'jan2022-1',
    'new-york-jan2022-2-of-10__2023-01-04_00-19-30_bqw/ITERS/it.5/5.events.csv.gz':'jan2022-2',
    'new-york-jan2022-3-of-10__2023-01-04_00-19-36_noe/ITERS/it.5/5.events.csv.gz':'jan2022-3',
    'new-york-jan2022-4-of-10__2023-01-04_00-19-33_nop/ITERS/it.5/5.events.csv.gz':'jan2022-4',
    'new-york-jan2022-5-of-10__2023-01-04_00-19-37_pnp/ITERS/it.5/5.events.csv.gz':'jan2022-5',
    'new-york-jan2022-6-of-10__2023-01-04_00-19-29_kny/ITERS/it.5/5.events.csv.gz':'jan2022-6',
    'new-york-jan2022-7-of-10__2023-01-04_00-19-39_qwt/ITERS/it.5/5.events.csv.gz':'jan2022-7',
    'new-york-jan2022-8-of-10__2023-01-04_00-19-35_fzq/ITERS/it.5/5.events.csv.gz':'jan2022-8',
    'new-york-jan2022-9-of-10__2023-01-04_00-19-31_dgg/ITERS/it.5/5.events.csv.gz':'jan2022-9',
}
data_names = data.keys()
pop_names = []
hs_names = []
for data_name in data_names:
    pop_names.append(data_name[:-26]+'population.csv.gz')
    hs_names.append(data_name[:-26]+'households.csv.gz')
names = data.values()

Merged_Pop = []
Merged_Hs = []
for name, pop_name, hs_name in zip(names, pop_names, hs_names):
    Merged_Pop.append(pd.read_csv(fp+pop_name)) 
    Merged_Hs.append(pd.read_csv(fp+hs_name))
Merged_Pop = pd.concat(Merged_Pop)
Merged_Hs = pd.concat(Merged_Hs)
Merged_Pop.to_csv(fp_res+'Merged_Pop_{}.csv.gz'.format(output_nm))
Merged_Hs.to_csv(fp_res+'Merged_Hs_{}.csv.gz'.format(output_nm))
print(len(Merged_Pop))
print(len(Merged_Hs))

In [ ]:
fp = "s3://beam-outputs/output/newyork/"
output_nm = 'May2020'
fp_res = 'outputs/'

data = {
    'new-york-may2020-0-of-10__2023-01-12_18-51-34_sju/ITERS/it.5/5.events.csv.gz':'may2020-0',
    'new-york-may2020-1-of-10__2023-01-12_18-51-33_nyd/ITERS/it.5/5.events.csv.gz':'may2020-1',
    'new-york-may2020-2-of-10__2023-01-12_18-51-29_yow/ITERS/it.5/5.events.csv.gz':'may2020-2',
    'new-york-may2020-3-of-10__2023-01-12_18-51-36_inx/ITERS/it.5/5.events.csv.gz':'may2020-3',
    'new-york-may2020-4-of-10__2023-01-12_18-51-31_iwm/ITERS/it.5/5.events.csv.gz':'may2020-4',
    'new-york-may2020-5-of-10__2023-01-12_18-51-34_jui/ITERS/it.5/5.events.csv.gz':'may2020-5',
    'new-york-may2020-6-of-10__2023-01-12_18-51-34_gqr/ITERS/it.5/5.events.csv.gz':'may2020-6',
    'new-york-may2020-7-of-10__2023-01-12_18-51-28_juu/ITERS/it.5/5.events.csv.gz':'may2020-7',
    'new-york-may2020-8-of-10__2023-01-12_18-51-36_ehv/ITERS/it.5/5.events.csv.gz':'may2020-8',
    'new-york-may2020-9-of-10__2023-01-12_18-51-54_bas/ITERS/it.5/5.events.csv.gz':'may2020-9',
}
data_names = data.keys()
pop_names = []
hs_names = []
for data_name in data_names:
    pop_names.append(data_name[:-26]+'population.csv.gz')
    hs_names.append(data_name[:-26]+'households.csv.gz')
names = data.values()

Merged_Pop = []
Merged_Hs = []
for name, pop_name, hs_name in zip(names, pop_names, hs_names):
    Merged_Pop.append(pd.read_csv(fp+pop_name)) 
    Merged_Hs.append(pd.read_csv(fp+hs_name))
Merged_Pop = pd.concat(Merged_Pop)
Merged_Hs = pd.concat(Merged_Hs)
Merged_Pop.to_csv(fp_res+'Merged_Pop_{}.csv.gz'.format(output_nm))
Merged_Hs.to_csv(fp_res+'Merged_Hs_{}.csv.gz'.format(output_nm))
print(len(Merged_Pop))
print(len(Merged_Hs))

In [ ]:
fp = "s3://beam-outputs/output/newyork/"
output_nm = 'May2022'
fp_res = 'outputs/'

data = {
    'new-york-june2022-0-of-10__2023-01-10_16-53-18_iir/ITERS/it.5/5.events.csv.gz':'may2022-0',
    'new-york-june2022-1-of-10__2023-01-10_16-53-23_mrs/ITERS/it.0/0.events.csv.gz':'may2022-1',
    'new-york-june2022-2-of-10__2023-01-10_16-53-36_log/ITERS/it.5/5.events.csv.gz':'may2022-2',
    'new-york-june2022-3-of-10__2023-01-10_16-53-36_lvd/ITERS/it.0/0.events.csv.gz':'may2022-3',
    'new-york-june2022-4-of-10__2023-01-10_16-53-44_afe/ITERS/it.0/0.events.csv.gz':'may2022-4',
    'new-york-june2022-5-of-10__2023-01-10_16-53-58_gry/ITERS/it.5/5.events.csv.gz':'may2022-5',
    'new-york-june2022-6-of-10__2023-01-10_16-53-59_qdi/ITERS/it.5/5.events.csv.gz':'may2022-6',
    'new-york-june2022-7-of-10__2023-01-10_16-54-13_fwj/ITERS/it.0/0.events.csv.gz':'may2022-7',
    'new-york-june2022-8-of-10__2023-01-10_16-54-18_okl/ITERS/it.5/5.events.csv.gz':'may2022-8',
    'new-york-june2022-9-of-10__2023-01-10_16-54-38_jvb/ITERS/it.5/5.events.csv.gz':'may2022-9',
}
data_names = data.keys()
pop_names = []
hs_names = []
for data_name in data_names:
    pop_names.append(data_name[:-26]+'population.csv.gz')
    hs_names.append(data_name[:-26]+'households.csv.gz')
names = data.values()

Merged_Pop = []
Merged_Hs = []
for name, pop_name, hs_name in zip(names, pop_names, hs_names):
    Merged_Pop.append(pd.read_csv(fp+pop_name)) 
    Merged_Hs.append(pd.read_csv(fp+hs_name))
Merged_Pop = pd.concat(Merged_Pop)
Merged_Hs = pd.concat(Merged_Hs)
Merged_Pop.to_csv(fp_res+'Merged_Pop_{}.csv.gz'.format(output_nm))
Merged_Hs.to_csv(fp_res+'Merged_Hs_{}.csv.gz'.format(output_nm))
print(len(Merged_Pop))
print(len(Merged_Hs))

In [ ]:
fp = "s3://beam-outputs/output/newyork/"
output_nm = 'Aug2021'
fp_res = 'outputs/'

data = {
    'new-york-august2021-0-of-10__2023-01-12_18-51-43_rxk/ITERS/it.5/5.events.csv.gz':'aug2021-0',
    'new-york-august2021-1-of-10__2023-01-12_18-51-29_aza/ITERS/it.5/5.events.csv.gz':'aug2021-1',
    'new-york-august2021-2-of-10__2023-01-12_18-51-33_xqs/ITERS/it.5/5.events.csv.gz':'aug2021-2',
    'new-york-august2021-3-of-10__2023-01-12_18-51-29_abg/ITERS/it.5/5.events.csv.gz':'aug2021-3',
    'new-york-august2021-4-of-10__2023-01-12_18-51-34_lwg/ITERS/it.5/5.events.csv.gz':'aug2021-4',
    'new-york-august2021-5-of-10__2023-01-12_18-51-34_air/ITERS/it.5/5.events.csv.gz':'aug2021-5',
    'new-york-august2021-6-of-10__2023-01-12_18-51-28_vbj/ITERS/it.5/5.events.csv.gz':'aug2021-6',
    'new-york-august2021-7-of-10__2023-01-12_18-51-29_dnu/ITERS/it.5/5.events.csv.gz':'aug2021-7',
    'new-york-august2021-8-of-10__2023-01-12_18-51-29_uzv/ITERS/it.5/5.events.csv.gz':'aug2021-8',
    'new-york-august2021-9-of-10__2023-01-19_00-58-43_fwc/ITERS/it.5/5.events.csv.gz':'aug2021-9',
}
data_names = data.keys()
pop_names = []
hs_names = []
for data_name in data_names:
    pop_names.append(data_name[:-26]+'population.csv.gz')
    hs_names.append(data_name[:-26]+'households.csv.gz')
names = data.values()

Merged_Pop = []
Merged_Hs = []
for name, pop_name, hs_name in zip(names, pop_names, hs_names):
    Merged_Pop.append(pd.read_csv(fp+pop_name)) 
    Merged_Hs.append(pd.read_csv(fp+hs_name))
Merged_Pop = pd.concat(Merged_Pop)
Merged_Hs = pd.concat(Merged_Hs)
Merged_Pop.to_csv(fp_res+'Merged_Pop_{}.csv.gz'.format(output_nm))
Merged_Hs.to_csv(fp_res+'Merged_Hs_{}.csv.gz'.format(output_nm))
print(len(Merged_Pop))
print(len(Merged_Hs))

In [ ]:
fp = "s3://beam-outputs/output/newyork/"
output_nm = 'Aug2020'
fp_res = 'outputs/'

data = {
    'new-york-august2020-0-of-10__2023-01-10_16-52-48_lgr/ITERS/it.5/5.events.csv.gz':'aug2020-0',
    'new-york-august2020-1-of-10__2023-01-10_16-52-46_gib/ITERS/it.5/5.events.csv.gz':'aug2020-1',
    'new-york-august2020-2-of-10__2023-01-14_21-19-35_flk/ITERS/it.5/5.events.csv.gz':'aug2020-2',
    'new-york-august2020-3-of-10__2023-01-15_01-54-44_txq/ITERS/it.5/5.events.csv.gz':'aug2020-3',
    'new-york-august2020-4-of-10__2023-01-10_16-52-43_rio/ITERS/it.5/5.events.csv.gz':'aug2020-4',
    'new-york-august2020-5-of-10__2023-01-10_16-53-00_vjr/ITERS/it.5/5.events.csv.gz':'aug2020-5',
    'new-york-august2020-6-of-10__2023-01-10_16-52-53_bhh/ITERS/it.5/5.events.csv.gz':'aug2020-6',
    'new-york-august2020-7-of-10__2023-01-10_16-53-01_drt/ITERS/it.5/5.events.csv.gz':'aug2020-7',
    'new-york-august2020-8-of-10__2023-01-10_16-53-15_qgy/ITERS/it.5/5.events.csv.gz':'aug2020-8',
    'new-york-august2020-9-of-10__2023-01-10_16-53-14_pdt/ITERS/it.5/5.events.csv.gz':'aug2020-9',
}
data_names = data.keys()
pop_names = []
hs_names = []
for data_name in data_names:
    pop_names.append(data_name[:-26]+'population.csv.gz')
    hs_names.append(data_name[:-26]+'households.csv.gz')
names = data.values()

Merged_Pop = []
Merged_Hs = []
for name, pop_name, hs_name in zip(names, pop_names, hs_names):
    Merged_Pop.append(pd.read_csv(fp+pop_name)) 
    Merged_Hs.append(pd.read_csv(fp+hs_name))
Merged_Pop = pd.concat(Merged_Pop)
Merged_Hs = pd.concat(Merged_Hs)
Merged_Pop.to_csv(fp_res+'Merged_Pop_{}.csv.gz'.format(output_nm))
Merged_Hs.to_csv(fp_res+'Merged_Hs_{}.csv.gz'.format(output_nm))
print(len(Merged_Pop))
print(len(Merged_Hs))

In [ ]:
#CREATE A TABLE WITH VEHICLE CAPACITIES FOR EACH SCENARIO

scenarios = ['new-york-baseline-0-of-10__2023-01-03_19-59-12_nwn/ITERS/it.5/5.events.csv.gz',
            'new-york-jan2022-0-of-10__2023-01-04_00-19-31_leg/ITERS/it.5/5.events.csv.gz',
            'new-york-may2020-0-of-10__2023-01-12_18-51-34_sju/ITERS/it.5/5.events.csv.gz',
            'new-york-june2022-0-of-10__2023-01-10_16-53-18_iir/ITERS/it.5/5.events.csv.gz',
            'new-york-august2021-0-of-10__2023-01-12_18-51-43_rxk/ITERS/it.5/5.events.csv.gz',
            'new-york-august2020-0-of-10__2023-01-10_16-52-48_lgr/ITERS/it.5/5.events.csv.gz',
            ]
names = ['Baseline2019','Jan2022','May2020','May2022', 'Aug2021','Aug2020']

for scenario, name in zip(scenarios, names):
    print('Read {}'.format(scenario))
    Pts_scenario  = pd.read_csv('s3://beam-outputs/output/newyork/{}'.format(scenario), nrows = None)
    Pts_scenario = Pts_scenario[(Pts_scenario['type']=='PathTraversal')]
    Pts_scenario = Pts_scenario[(Pts_scenario['mode']=='bus')|(Pts_scenario['mode']=='subway')|(Pts_scenario['mode']=='rail')|(Pts_scenario['mode']=='tram')|(Pts_scenario['mode']=='ferry')]
    vehicles = Pts_scenario.groupby(['vehicle','capacity']).apply(lambda x: int(np.mean(list(x.capacity))*10))
    
    vehicles.to_csv('outputs/vehicles_{}.csv'.format(name))
    # nuovo.to_csv('outputs/MergedPTs_NYC_{}_3.csv.gz'.format(scenario))


In [ ]:
# ADD YOUNG PEOPLE baseline

PTs = pd.read_csv('outputs/MergedPTs_NYC_Baseline2019_3.csv.gz', index_col=0)
PtoPts = pd.read_csv('outputs/MergedPtoPTs_NYC_Baseline2019_3.csv.gz', index_col=0)
persons = pd.read_csv('outputs/Merged_Pop_Baseline.csv.gz', index_col=0)
households = pd.read_csv('outputs/Merged_Hs_Baseline.csv.gz', index_col=0)
# persons_ip = pd.read_csv('outputs/persons.csv.gz', index_col=0)
# households_ip = pd.read_csv('outputs/households.csv.gz', index_col=0)
# persons_ip_allages = pd.read_csv('persons.csv.gz', index_col=0)
young_file = pd.read_csv('inputs/nyc-kids-base-fixed.csv', index_col=0)
vehicles = pd.read_csv('outputs/vehicles_Baseline2019.csv', index_col=0)
vehicles = dict(zip(vehicles.index, vehicles.capacity*10))

PTs['mode'] = PTs['mode'].replace('bus_empty', 'bus')
PTs['mode'] = PTs['mode'].replace('subway_empty', 'subway')
PTs['mode'] = PTs['mode'].replace('rail_empty', 'rail')
PTs['mode'] = PTs['mode'].replace('tram_empty', 'tram')
PTs['mode'] = PTs['mode'].replace('ferry_empty', 'ferry')

i =0
PtoPts_to_add = pd.DataFrame()
for adult_to_duplicate, young_personid, hsID, age, is_fem, ind in zip(young_file['adultPersonIdToCopy'],young_file.index, young_file['household_id'],young_file['age'],young_file['sex'],young_file['industry'],):
    if i%100 ==0:
        young_file['adultPersonIdToCopy'].to_csv('outputs/{}.csv'.format(i))
    PtoPts_to_duplicate = PtoPts[PtoPts['personID']==adult_to_duplicate]
    PtoPts_to_duplicate.loc[:,'personID'] = young_personid
    PtoPts_to_add = pd.concat([PtoPts_to_add, PtoPts_to_duplicate])
    persons.loc[99999999999+i,['personId','age','isFemale','householdId','householdRank','excludedModes','valueOfTime']] = [int(young_personid), int(age), True if is_fem==2 else False, hsID, 0, np.nan, 7.25]
    if str(hsID) not in list(households['householdId']):
        print('Warning, {} new household'.format(hsID))
        young_file['adultPersonIdToCopy'].to_csv('outputs/{}.csv'.format(hsID))
    i+=1
    for PT_id, person_id in zip(PtoPts_to_duplicate['pathTraversalID'],PtoPts_to_duplicate['personID']):
        PTs.loc[PT_id,'occupancy'] +=1
        PTs.loc[PT_id,'riders'] += ':{}'.format(person_id)
        PTs.loc[PT_id,'passengerMiles'] += PTs.loc[PT_id,'vehicleMiles']

PtoPts = pd.concat([PtoPts, PtoPts_to_add])
persons.index = range(len(persons))
PtoPts.index = range(len(PtoPts))


persons['personId'] = persons.personId.astype(int)
persons['householdRank'] = persons.householdRank.astype(int)

capacities = []
for vehicle in PTs['vehicle']:
    capacities.append(vehicles[vehicle])
PTs['capacity'] = capacities

PtoPts.to_csv('outputs/MergedPtoPTs_NYC_Baseline2019_4.csv.gz')
persons.to_csv('outputs/Merged_Pop_Baseline_4.csv.gz')
PTs.to_csv('outputs/MergedPTs_NYC_Baseline2019_4.csv.gz')

In [ ]:
# ADD YOUNG PEOPLE aug2020

PTs = pd.read_csv('outputs/MergedPTs_NYC_Aug2020_3.csv.gz', index_col=0)
PtoPts = pd.read_csv('outputs/MergedPtoPTs_NYC_Aug2020_3.csv.gz', index_col=0)
persons = pd.read_csv('outputs/Merged_Pop_Aug2020.csv.gz', index_col=0)
households = pd.read_csv('outputs/Merged_Hs_Aug2020.csv.gz', index_col=0)
# persons_ip = pd.read_csv('outputs/persons.csv.gz', index_col=0)
# households_ip = pd.read_csv('outputs/households.csv.gz', index_col=0)
# persons_ip_allages = pd.read_csv('persons.csv.gz', index_col=0)
young_file = pd.read_csv('inputs/nyc-kids-august2020.csv', index_col=0)
vehicles = pd.read_csv('outputs/vehicles_Aug2020.csv', index_col=0)
vehicles = dict(zip(vehicles.index, vehicles.capacity*10))

PTs['mode'] = PTs['mode'].replace('bus_empty', 'bus')
PTs['mode'] = PTs['mode'].replace('subway_empty', 'subway')
PTs['mode'] = PTs['mode'].replace('rail_empty', 'rail')
PTs['mode'] = PTs['mode'].replace('tram_empty', 'tram')
PTs['mode'] = PTs['mode'].replace('ferry_empty', 'ferry')

i =0
PtoPts_to_add = pd.DataFrame()
for adult_to_duplicate, young_personid, hsID, age, is_fem, ind in zip(young_file['adultPersonIdToCopy'],young_file.index, young_file['household_id'],young_file['age'],young_file['sex'],young_file['industry'],):
    if i%100 ==0:
        young_file['adultPersonIdToCopy'].to_csv('outputs/{}.csv'.format(i))
    PtoPts_to_duplicate = PtoPts[PtoPts['personID']==adult_to_duplicate]
    PtoPts_to_duplicate.loc[:,'personID'] = young_personid
    PtoPts_to_add = pd.concat([PtoPts_to_add, PtoPts_to_duplicate])
    persons.loc[99999999999+i,['personId','age','isFemale','householdId','householdRank','excludedModes','valueOfTime']] = [int(young_personid), int(age), True if is_fem==2 else False, hsID, 0, np.nan, 7.25]
    if str(hsID) not in list(households['householdId']):
        print('Warning, {} new household'.format(hsID))
        young_file['adultPersonIdToCopy'].to_csv('outputs/{}.csv'.format(hsID))
    i+=1
    for PT_id, person_id in zip(PtoPts_to_duplicate['pathTraversalID'],PtoPts_to_duplicate['personID']):
        PTs.loc[PT_id,'occupancy'] +=1
        PTs.loc[PT_id,'riders'] += ':{}'.format(person_id)
        PTs.loc[PT_id,'passengerMiles'] += PTs.loc[PT_id,'vehicleMiles']

PtoPts = pd.concat([PtoPts, PtoPts_to_add])
persons.index = range(len(persons))
PtoPts.index = range(len(PtoPts))


persons['personId'] = persons.personId.astype(int)
persons['householdRank'] = persons.householdRank.astype(int)

capacities = []
for vehicle in PTs['vehicle']:
    capacities.append(vehicles[vehicle])
PTs['capacity'] = capacities

PtoPts.to_csv('outputs/MergedPtoPTs_NYC_Aug2020_4.csv.gz')
persons.to_csv('outputs/Merged_Pop_Aug2020_4.csv.gz')
PTs.to_csv('outputs/MergedPTs_NYC_Aug2020_4.csv.gz')

In [2]:
# CREATE PERSON TABLE
Pts = pd.read_csv('outputs/MergedPTs_NYC_Aug2020_4.csv.gz', index_col=0)
PtoPts = pd.read_csv('outputs/MergedPtoPTs_NYC_Aug2020_4.csv.gz', index_col=0)
Persons = pd.read_csv('outputs/Merged_Pop_Aug2020_4.csv.gz', index_col=0)
Households = pd.read_csv('outputs/Merged_Hs_Aug2020.csv.gz', index_col=0)
pop = pd.read_csv('outputs/persons_baseline.csv.gz', index_col=0)
hs = pd.read_csv('outputs/households_baseline.csv.gz', index_col=0)

In [ ]:

print('s')
# plans_baseline1 =     pd.read_csv('s3://beam-outputs/output/newyork/new-york-baseline-0-of-10__2023-01-03_19-59-12_nwn/ITERS/it.5/5.plans.csv.gz', usecols = ['personId','planSelected','activityType', 'activityLocationX', 'activityLocationY', 'activityStartTime', 'activityEndTime'])
# print('s')

# plans_baseline2 =     pd.read_csv('s3://beam-outputs/output/newyork/new-york-baseline-1-of-10__2023-01-03_19-59-13_jgp/ITERS/it.5/5.plans.csv.gz', usecols = ['personId','planSelected','activityType', 'activityLocationX', 'activityLocationY', 'activityStartTime', 'activityEndTime'])

# print('s')
# plans_baseline3 =     pd.read_csv('s3://beam-outputs/output/newyork/new-york-baseline-2-of-10__2023-01-03_19-59-06_brm/ITERS/it.5/5.plans.csv.gz', usecols = ['personId','planSelected','activityType', 'activityLocationX', 'activityLocationY', 'activityStartTime', 'activityEndTime'])
# print('s')
# plans_baseline4 =     pd.read_csv('s3://beam-outputs/output/newyork/new-york-baseline-3-of-10__2023-01-03_19-59-07_kqr/ITERS/it.5/5.plans.csv.gz', usecols = ['personId','planSelected','activityType', 'activityLocationX', 'activityLocationY', 'activityStartTime', 'activityEndTime'])

# print('s')
# plans_baseline5 =     pd.read_csv('s3://beam-outputs/output/newyork/new-york-baseline-4-of-10__2023-01-03_19-59-09_fbb/ITERS/it.5/5.plans.csv.gz', usecols = ['personId','planSelected','activityType', 'activityLocationX', 'activityLocationY', 'activityStartTime', 'activityEndTime'])

# print('s')
# plans_baseline6 =     pd.read_csv('s3://beam-outputs/output/newyork/new-york-baseline-5-of-10__2023-01-03_19-59-11_tjh/ITERS/it.5/5.plans.csv.gz', usecols = ['personId','planSelected','activityType', 'activityLocationX', 'activityLocationY', 'activityStartTime', 'activityEndTime'])

# print('s')
# plans_baseline7 =     pd.read_csv('s3://beam-outputs/output/newyork/new-york-baseline-6-of-10__2023-01-03_19-59-12_zwm/ITERS/it.5/5.plans.csv.gz', usecols = ['personId','planSelected','activityType', 'activityLocationX', 'activityLocationY', 'activityStartTime', 'activityEndTime'])

# print('s')
# plans_baseline8 =     pd.read_csv('s3://beam-outputs/output/newyork/new-york-baseline-7-of-10__2023-01-03_19-59-19_bcr/ITERS/it.5/5.plans.csv.gz', usecols = ['personId','planSelected','activityType', 'activityLocationX', 'activityLocationY', 'activityStartTime', 'activityEndTime'])

# print('s')
# plans_baseline9 =     pd.read_csv('s3://beam-outputs/output/newyork/new-york-baseline-8-of-10__2023-01-03_19-59-14_olx/ITERS/it.5/5.plans.csv.gz', usecols = ['personId','planSelected','activityType', 'activityLocationX', 'activityLocationY', 'activityStartTime', 'activityEndTime'])

# print('s')
# plans_baseline0 =     pd.read_csv('s3://beam-outputs/output/newyork/new-york-baseline-9-of-10__2023-01-03_19-59-07_xcc/ITERS/it.5/5.plans.csv.gz', usecols = ['personId','planSelected','activityType', 'activityLocationX', 'activityLocationY', 'activityStartTime', 'activityEndTime'])
# print('s')


plans_baseline0 =     pd.read_csv('s3://beam-outputs/output/newyork/new-york-august2020-0-of-10__2023-01-10_16-52-48_lgr/ITERS/it.5/5.plans.csv.gz', usecols = ['personId','planSelected','activityType', 'activityLocationX', 'activityLocationY', 'activityStartTime', 'activityEndTime'])
plans_baseline1 =     pd.read_csv('s3://beam-outputs/output/newyork/new-york-august2020-1-of-10__2023-01-10_16-52-46_gib/ITERS/it.5/5.plans.csv.gz', usecols = ['personId','planSelected','activityType', 'activityLocationX', 'activityLocationY', 'activityStartTime', 'activityEndTime'])
plans_baseline2 =     pd.read_csv('s3://beam-outputs/output/newyork/new-york-august2020-2-of-10__2023-01-14_21-19-35_flk/ITERS/it.5/5.plans.csv.gz', usecols = ['personId','planSelected','activityType', 'activityLocationX', 'activityLocationY', 'activityStartTime', 'activityEndTime'])
plans_baseline3 =     pd.read_csv('s3://beam-outputs/output/newyork/new-york-august2020-3-of-10__2023-01-15_01-54-44_txq/ITERS/it.5/5.plans.csv.gz', usecols = ['personId','planSelected','activityType', 'activityLocationX', 'activityLocationY', 'activityStartTime', 'activityEndTime'])
plans_baseline4 =     pd.read_csv('s3://beam-outputs/output/newyork/new-york-august2020-4-of-10__2023-01-10_16-52-43_rio/ITERS/it.5/5.plans.csv.gz', usecols = ['personId','planSelected','activityType', 'activityLocationX', 'activityLocationY', 'activityStartTime', 'activityEndTime'])
plans_baseline5 =     pd.read_csv('s3://beam-outputs/output/newyork/new-york-august2020-5-of-10__2023-01-10_16-53-00_vjr/ITERS/it.5/5.plans.csv.gz', usecols = ['personId','planSelected','activityType', 'activityLocationX', 'activityLocationY', 'activityStartTime', 'activityEndTime'])
plans_baseline6 =     pd.read_csv('s3://beam-outputs/output/newyork/new-york-august2020-6-of-10__2023-01-10_16-52-53_bhh/ITERS/it.5/5.plans.csv.gz', usecols = ['personId','planSelected','activityType', 'activityLocationX', 'activityLocationY', 'activityStartTime', 'activityEndTime'])
plans_baseline7 =     pd.read_csv('s3://beam-outputs/output/newyork/new-york-august2020-7-of-10__2023-01-10_16-53-01_drt/ITERS/it.5/5.plans.csv.gz', usecols = ['personId','planSelected','activityType', 'activityLocationX', 'activityLocationY', 'activityStartTime', 'activityEndTime'])
plans_baseline8 =     pd.read_csv('s3://beam-outputs/output/newyork/new-york-august2020-8-of-10__2023-01-10_16-53-15_qgy/ITERS/it.5/5.plans.csv.gz', usecols = ['personId','planSelected','activityType', 'activityLocationX', 'activityLocationY', 'activityStartTime', 'activityEndTime'])
plans_baseline9 =     pd.read_csv('s3://beam-outputs/output/newyork/new-york-august2020-9-of-10__2023-01-10_16-53-14_pdt/ITERS/it.5/5.plans.csv.gz', usecols = ['personId','planSelected','activityType', 'activityLocationX', 'activityLocationY', 'activityStartTime', 'activityEndTime'])


s


In [ ]:
plans_baseline = pd.concat([plans_baseline0,plans_baseline1,plans_baseline2,plans_baseline3,plans_baseline4,
                            plans_baseline5,plans_baseline6,plans_baseline7,plans_baseline8,plans_baseline9])

In [ ]:
i=0
durations = []
modes = []
print(len(PtoPts))
for ptid in PtoPts['pathTraversalID']:
    durations.append(Pts.loc[ptid, 'duration'])
    modes.append(Pts.loc[ptid, 'mode'])
    i+=1
    if i%100000==0:
        print(i)
PtoPts['duration']=durations
PtoPts['mode']=modes

agents = list(np.unique(list(PtoPts['personID'])))



In [ ]:
PtoPts_gropued = PtoPts.groupby(['personID','mode']).apply(lambda x: [sum(x.duration),len(x), len(np.unique(list(x.vehicleID))),list(np.unique(list(x.planIndex)))])


In [ ]:
PtoPts['mode'] = PtoPts['mode'].replace('bus_empty', 'bus')
PtoPts['mode'] = PtoPts['mode'].replace('subway_empty', 'subway')
PtoPts['mode'] = PtoPts['mode'].replace('rail_empty', 'rail')
PtoPts['mode'] = PtoPts['mode'].replace('tram_empty', 'tram')
PtoPts['mode'] = PtoPts['mode'].replace('ferry_empty', 'ferry')

Persons['personId'] = Persons.personId.astype(int)
Persons['householdRank'] = Persons.householdRank.astype(int)


In [ ]:
person_database = pd.DataFrame()

person_database['PersonID'] = agents



person_to_ages = dict(zip(pop.index, pop.age))
person_to_gender = dict(zip(pop.index, pop.sex))
person_to_industry = dict(zip(pop.index, pop.industry))

print('s')

plans_baseline_grouped = plans_baseline.groupby(['personId','planSelected','activityType']).apply(lambda x: [list(x.activityLocationX)[0],list( x.activityLocationY)[0], list(x.activityStartTime)[0], list(x.activityEndTime)[0]])

print('s')

In [ ]:
ages = []
is_female = []
industry = []
residency_x = []
residency_y = []
work_x = []
work_y = []
work_start = []
work_end = []
i=0
print(len(agents))
wrong_home=0
wrong_work=0
for agent in person_database['PersonID']:
    if i%100000==0:
        print(i)
    i+=1
    ages.append(person_to_ages[agent])
    is_female.append(person_to_gender[agent])
    industry.append(person_to_industry[agent])
    try:
        residency_x.append(plans_baseline_grouped[(agent,True,'Home')][0])
        residency_y.append(plans_baseline_grouped[(agent,True,'Home')][1])
    except:
        residency_x.append(0)
        residency_y.append(0)
        wrong_home+=1
    try:
        work_x.append(plans_baseline_grouped[(agent,True,'Work')][0])
        work_y.append(plans_baseline_grouped[(agent,True,'Work')][1])
        work_start.append(plans_baseline_grouped[(agent,True,'Work')][2])
        work_end.append(plans_baseline_grouped[(agent,True,'Work')][3])
    except:
        work_x.append(0)
        work_y.append(0)
        work_start.append(0)
        work_end.append(0)
        wrong_work+=1
print('wrong_home',wrong_home)
print('wrong_work',wrong_work)
        
print('s')

person_database['Age'] = ages
person_database['Gender'] = is_female
person_database['Industry'] = industry
person_database['X_Home'] = residency_x
person_database['Y_Home'] = residency_y
person_database['X_Work'] = work_x
person_database['Y_Work'] = work_y

In [ ]:


time_spent_on_bus = []
time_spent_on_sub = []
time_spent_on_rail = []
time_spent_on_tram =[]
time_spent_on_ferry = []

total_time_on_transit = []

legs_on_bus = []
legs_on_sub = []
legs_on_rail = []
legs_on_tram = []
legs_on_ferry = []

total_legs_on_transit = []

bus_used_in_a_day =[]
sub_used_in_a_day =[]
rail_used_in_a_day =[]
tram_used_in_a_day = []
ferry_used_in_a_day = []

total_transit_vehicles_used = []

n_transfer = []


    
i=0
for agent in agents:
    
    i+=1
    if i%100000==0:
        print(i)
        
    try:
        time_spent_on_bus.append(PtoPts_gropued[agent,'bus'][0])
    except:
        time_spent_on_bus.append(0)
    try:
        time_spent_on_sub.append(PtoPts_gropued[agent,'subway'][0])
    except:
        time_spent_on_sub.append(0)
    try:
        time_spent_on_rail.append(PtoPts_gropued[agent,'rail'][0])
    except:
        time_spent_on_rail.append(0)
    try:
        time_spent_on_tram.append(PtoPts_gropued[agent,'tram'][0])
    except:
        time_spent_on_tram.append(0)
    try:
        time_spent_on_ferry.append(PtoPts_gropued[agent,'ferry'][0])
    except:
        time_spent_on_ferry.append(0)
        
        
    try:
        legs_on_bus.append(PtoPts_gropued[agent,'bus'][1])
    except:
        legs_on_bus.append(0)
    try:
        legs_on_sub.append(PtoPts_gropued[agent,'subway'][1])
    except:
        legs_on_sub.append(0)
    try:
        legs_on_rail.append(PtoPts_gropued[agent,'rail'][1])
    except:
        legs_on_rail.append(0)
    try:
        legs_on_tram.append(PtoPts_gropued[agent,'tram'][1])
    except:
        legs_on_tram.append(0)
    try:
        legs_on_ferry.append(PtoPts_gropued[agent,'ferry'][1])
    except:
        legs_on_ferry.append(0)
        
    try:
        bus_used_in_a_day.append(PtoPts_gropued[agent,'bus'][2])
    except:
        bus_used_in_a_day.append(0)
    try:
        sub_used_in_a_day.append(PtoPts_gropued[agent,'subway'][2])
    except:
        sub_used_in_a_day.append(0)
    try:
        rail_used_in_a_day.append(PtoPts_gropued[agent,'rail'][2])
    except:
        rail_used_in_a_day.append(0)
    try:
        tram_used_in_a_day.append(PtoPts_gropued[agent,'tram'][2])
    except:
        tram_used_in_a_day.append(0)
    try:
        ferry_used_in_a_day.append(PtoPts_gropued[agent,'ferry'][2])
    except:
        ferry_used_in_a_day.append(0)
        
        
    total_time_on_transit.append(time_spent_on_bus[-1]+time_spent_on_sub[-1]+time_spent_on_rail[-1]+time_spent_on_tram[-1]+ time_spent_on_ferry[-1])
    
    total_legs_on_transit.append(legs_on_bus[-1]+legs_on_sub[-1]+legs_on_rail[-1]+legs_on_tram[-1]+ legs_on_ferry[-1] )
    
    total_transit_vehicles_used.append(bus_used_in_a_day[-1]+sub_used_in_a_day[-1]+rail_used_in_a_day[-1]+tram_used_in_a_day[-1]+ ferry_used_in_a_day[-1]  )
    
    n_transfer.append(total_transit_vehicles_used[-1] - len(np.unique([item for sublist in [PtoPts_gropued[agent,mode_][3]  for mode_ in PtoPts_gropued[agent].keys() ] for item in sublist])))

    
    

person_database['Time spent on bus'] = time_spent_on_bus
person_database['Time spent on subway'] = time_spent_on_sub
person_database['Time spent on rail'] = time_spent_on_rail
person_database['Time spent on tram'] = time_spent_on_tram
person_database['Time spent on ferry'] = time_spent_on_ferry
        
        
person_database['Number of bus stops traveled'] = legs_on_bus
person_database['Number of subway stops traveled'] = legs_on_sub
person_database['Number of rail stops traveled'] = legs_on_rail
person_database['Number of tram stops traveled'] = legs_on_tram
person_database['Number of ferry stops traveled'] = legs_on_ferry
        
person_database['Bus vehicles used'] = bus_used_in_a_day
person_database['Subway vehicles used'] = sub_used_in_a_day
person_database['Rail vehicles used'] = rail_used_in_a_day
person_database['Tram vehicles used'] = tram_used_in_a_day
person_database['Ferry vehicles used'] = ferry_used_in_a_day
        
        
person_database['Time spent on transit'] = total_time_on_transit
    
person_database['Number of transit stops traveled'] = total_legs_on_transit
    
person_database['Transit vehicles used'] = total_transit_vehicles_used
    
person_database['Number of transfers'] = n_transfer




In [ ]:

person_database.to_csv('outputs/agents_experience_Aug2020.cvs')
# person_database['Number of transfers'] = n_transfer


In [ ]:
person_database[person_database['PersonID']==2188823]

In [ ]:
#PLOT AGENT IN A MAP

In [ ]:
Pts = pd.read_csv('outputs/MergedPTs_NYC_Baseline2019_4.csv.gz', index_col=0)
PtoPts = pd.read_csv('outputs/MergedPtoPTs_NYC_Baseline2019_4.csv.gz', index_col=0)


In [ ]:
Persons = pd.read_csv('outputs/Merged_Pop_Baseline_4.csv.gz', index_col=0)
Households = pd.read_csv('outputs/Merged_Hs_Baseline.csv.gz', index_col=0)

In [ ]:
Plans = pd.concat(
    [pd.read_csv('s3://beam-outputs/output/newyork/new-york-baseline-0-of-10__2023-01-03_19-59-12_nwn/ITERS/it.5/5.plans.csv.gz'),
    pd.read_csv('s3://beam-outputs/output/newyork/new-york-baseline-1-of-10__2023-01-03_19-59-13_jgp/ITERS/it.5/5.plans.csv.gz'),
    pd.read_csv('s3://beam-outputs/output/newyork/new-york-baseline-2-of-10__2023-01-03_19-59-06_brm/ITERS/it.5/5.plans.csv.gz'),
    pd.read_csv('s3://beam-outputs/output/newyork/new-york-baseline-3-of-10__2023-01-03_19-59-07_kqr/ITERS/it.5/5.plans.csv.gz'),
    pd.read_persons.csv.gz's3://beam-outputs/output/newyork/new-york-baseline-4-of-10__2023-01-03_19-59-09_fbb/ITERS/it.5/5.plans.csv.gz'),])
    # pd.read_csv('s3://beam-outputs/output/newyork/new-york-baseline-5-of-10__2023-01-03_19-59-11_tjh/ITERS/it.5/5.plans.csv.gz'),
    # pd.read_csv('s3://beam-outputs/output/newyork/new-york-baseline-6-of-10__2023-01-03_19-59-12_zwm/ITERS/it.5/5.plans.csv.gz'),
    # pd.read_csv('s3://beam-outputs/output/newyork/new-york-baseline-7-of-10__2023-01-03_19-59-19_bcr/ITERS/it.5/5.plans.csv.gz'),
    # pd.read_csv('s3://beam-outputs/output/newyork/new-york-baseline-8-of-10__2023-01-03_19-59-14_olx/ITERS/it.5/5.plans.csv.gz'),
    # pd.read_csv('s3://beam-outputs/output/newyork/new-york-baseline-9-of-10__2023-01-03_19-59-07_xcc/ITERS/it.5/5.plans.csv.gz')])

In [ ]:
PtoPts_user_NYC_Subway =  PtoPts[PtoPts['personID']==  PtoPts['personID'][PtoPts['agency'] == 'NYC_Subway'].value_counts().keys()[183]]
# user_NJ_Transit_Bus = PtoPts['personID'][PtoPts['agency'] == 'NJ_Transit_Bus'].value_counts().keys()[200]
# user_MTA_Brookl = PtoPts['personID'][PtoPts['agency'] == 'MTA_Brookl'].value_counts().keys()[200]
# user_NYC_Bus_Co = PtoPts['personID'][PtoPts['agency'] == 'NYC_Bus_Co'].value_counts().keys()[200]
# user_NICE_117_2 = PtoPts['personID'][PtoPts['agency'] == 'NICE_117_2'].value_counts().keys()[200]
# user_MTA_Manhat = PtoPts['personID'][PtoPts['agency'] == 'MTA_Manhat'].value_counts().keys()[200]
# user_MTA_Queens = PtoPts['personID'][PtoPts['agency'] == 'MTA_Queens'].value_counts().keys()[200]
# user_MTA_Bronx_ = PtoPts['personID'][PtoPts['agency'] == 'MTA_Bronx_'].value_counts().keys()[200]
# user_MTA_Staten = PtoPts['personID'][PtoPts['agency'] == 'MTA_Staten'].value_counts().keys()[200]
# user_NJ_Transit_Rail = PtoPts['personID'][PtoPts['agency'] == 'NJ_Transit_Rail'].value_counts().keys()[200]
# user_Long_Islan = PtoPts['personID'][PtoPts['agency'] == 'Long_Islan'].value_counts().keys()[200]
# user_PATH = PtoPts['personID'][PtoPts['agency'] == '151_631:t_'].value_counts().keys()[200]
# user_WCDOT_2020 = PtoPts['personID'][PtoPts['agency'] == 'WCDOT_2020'].value_counts().keys()[200]
# user_Metro_Nort = PtoPts['personID'][PtoPts['agency'] == 'etro-Nort'].value_counts().keys()[200]
# user_NYC_NYC_Ferry_ = PtoPts['personID'][PtoPts['agency'] == 'NYC_Ferry_'].value_counts().keys()[200]
# user_NYC_Staten_Isl = PtoPts['personID'][PtoPts['agency'] == 'Staten_Isl'].value_counts().keys()[200]
# user_NYC_JFK_Airtra = PtoPts['personID'][PtoPts['agency'] == 'JFK_Airtra'].value_counts().keys()[200]

In [ ]:
PtoPts_user_NYC_Subway.to_csv('outputs/example_trip.csv')

In [ ]:
Plans_person = Plans[(Plans['personId']==4611317)&(Plans['planSelected'] == True)]

In [ ]:
Plans_person

In [ ]:
transit_agencies = ['NJ_Transit_Rail', 
                    'NJ_Transit_Bus', 
                    '151_631:t_',
                    'Long_Islan',
                    'Metro-Nort',
                    'MTA_Bronx_', 
                    'MTA_Brookl', 
                    'MTA_Manhat', 
                    'MTA_Queens', 
                    'MTA_Staten',
                    'NYC_Bus_Co', 
                    'NYC_Ferry_', 
                    'NYC_Subway', 
                    'Staten_Island_Ferry',
                    #'Suffolk_Co', 
                    'WCDOT_2019']

In [ ]:
map_agents = pd.DataFrame()
xs = []
ys = []


agencies =[]
times = []
vehicles = []

for ptid, ag in zip(PtoPts_user_NYC_Subway['pathTraversalID'], PtoPts_user_NYC_Subway['agency']):
    xs.append(Pts.loc[ptid, 'startX'])
    ys.append(Pts.loc[ptid, 'startY'])
    xs.append(Pts.loc[ptid, 'endX'])
    ys.append(Pts.loc[ptid, 'endY'])
    agencies.append(ag)
    agencies.append(ag)
    times.append(Pts.loc[ptid, 'departureTime'])
    times.append(Pts.loc[ptid, 'departureTime'])
    vehicles.append(Pts.loc[ptid, 'vehicle'])
    vehicles.append(Pts.loc[ptid, 'vehicle'])
map_agents['X'] = xs
map_agents['Y'] = ys
map_agents['Agency'] = agencies
map_agents['Time'] = times
map_agents['Vehicles'] = vehicles

map_agents.to_csv('outputs/NYC_agents.csv')

In [ ]:
Plans_person.loc[[3557470,3557472],:]

In [ ]:
plan_agents = pd.DataFrame()
Plans_person = Plans_person.loc[[3557470,3557472],:]

xs = []
ys = []
atypes =[]
for x, y, atype in zip(Plans_person['activityLocationX'], Plans_person['activityLocationY'], Plans_person['activityType']):
    xs.append(x)
    ys.append(y)
    atypes.append(atype)
plan_agents['X'] = xs
plan_agents['Y'] = ys
plan_agents['Atype'] = atypes

plan_agents.to_csv('outputs/NYC_activities.csv')

In [ ]:
Pts

In [ ]:
events = pd.read_csv('s3://beam-outputs/pilates-outputs/sfbay_5fleets_100price_100fleet_20230209/beam/year-2020-iteration-3/ITERS/it.0/0.events.csv.gz', nrows = None)





In [ ]:
events['vehicle'].to_csv('outputs/test.csv')

In [ ]:
# events=events.fillna(0)
events = events[events['type']=='PathTraversal']
eventsRH = events[events['vehicle'].str.contains('ride')]
agencies = []
for vehicle in eventsRH['vehicle']:
    agencies.append(vehicle.split('@')[1])
eventsRH['agency'] = agencies


In [ ]:
eventsRH['agency'].value_counts()

In [ ]:
eventsRH['is_pool']=0
eventsRH['is_pool'][eventsRH['numPassengers']>0]+=1
eventsRH['is_pool'][eventsRH['numPassengers']>1]+=1
eventsRH['is_pool'] = eventsRH['is_pool'].replace(0, 'empty')
eventsRH['is_pool'] = eventsRH['is_pool'].replace(1, 'solo')
eventsRH['is_pool'] = eventsRH['is_pool'].replace(2, 'pooled')

eventsRH.groupby(['agency','is_pool']).apply(lambda x: len(x))


In [ ]:
eventsRH['is_pool'].value_counts()

In [ ]:
eventsRH['numPassengers'].value_counts()